In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [12]:
essence_url = []

In [13]:
for i in range (1,64):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/essence-serum.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    essence_list = soup.find_all('div', class_='product-item-info')
    for item in essence_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            essence_url.append(link['href'])
print(len(essence_url))

756


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [179]:
essence_url

[]

In [157]:
details = []

In [166]:
prices = []
names = []
best_seller = []
for i in range (1,64):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/essence-serum.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    try:
        name = soup.find_all('h2', class_="product-name")
        for title in name:
            names.append(title.text)
        bestseller = soup.find_all('div', class_='product-badge product-badge-bestseller')
        for images in bestseller:
            best_seller.append(images.find('img', alt='Best Sellers').get('alt'))
        price = soup.find_all('span', class_="regular-price")
        for euros in price:
            prices.append(euros.find('span', class_='price').text)
    except:
        names.append('no info')
        best_seller.append('no info')
        prices.append('no info')

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [1]:
prices

NameError: name 'prices' is not defined

In [14]:
dataframe = [] 
for i in range (1,3):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/essence-serum.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    print(r.status_code)
    try:
        name = soup.find_all('h2', class_="product-name")
        for title in name:
            name = title.text
        bestseller = soup.find_all('div', class_='product-badge product-badge-bestseller')
        for images in bestseller:
            bestseller = images.find('img', alt='Best Sellers').get('alt')
        price = soup.find_all('span', class_="regular-price")
        for euros in price:
            price = euros.find('span', class_='price').text
    except:
        name = 'no info'
        bestseller = 'no info'
        price = 'no info'
        
    info = {
        'product name': name,
        'cost': price,
        'stars': bestseller}
    
    dataframe.append(info)

200
200


'iUNIK - Beta Glucan Power Moisture Serum - 50ml'

In [11]:
names

'no info'

In [49]:
turl = f'https://www.stylevana.com/en_EU/skincare/face-care/essence-serum.html?p=1' 
r = requests.get(turl)
soup = BeautifulSoup(r.content, 'lxml')
print(r.status_code)
name = soup.find_all('h2', class_="product-name", title=True)
price = soup.find_all('span', class_="regular-price")

200


In [85]:
name = soup.find_all('h2', class_="product-name")

In [86]:
name

[<h2 class="product-name"><a href="https://www.stylevana.com/en_EU/iunik-tea-tree-relief-serum-50ml.html" title="iUNIK - Tea Tree Relief Serum - 50ml">iUNIK - Tea Tree Relief Serum - 50ml</a></h2>,
 <h2 class="product-name"><a href="https://www.stylevana.com/en_EU/cosrx-advanced-snail-96-mucin-power-essence-100ml.html" title="COSRX - Advanced Snail 96 Mucin Power Essence 100ml">COSRX - Advanced Snail 96 Mucin Power Essence 100ml</a></h2>,
 <h2 class="product-name"><a href="https://www.stylevana.com/en_EU/beauty-of-joseon-repair-serum-ginseng-snail-mucin-30ml.html" title="BEAUTY OF JOSEON - Repair Serum : Ginseng + Snail Mucin - 30ml">BEAUTY OF JOSEON - Repair Serum : Ginseng + Snail Mucin - 30ml</a></h2>,
 <h2 class="product-name"><a href="https://www.stylevana.com/en_EU/krave-great-barrier-relief-45ml-1.html" title="Krave - Great Barrier Relief - 45ml">Krave - Great Barrier Relief - 45ml</a></h2>,
 <h2 class="product-name"><a href="https://www.stylevana.com/en_EU/beauty-of-joseon-glow

In [145]:
best_seller = []
for i in range (1,3):
    url = f'https://www.stylevana.com/en_EU/skincare/face-care/essence-serum.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    print(r.status_code)
    bestseller = soup.find_all('div', class_='product-badge product-badge-bestseller')
    for images in bestseller:
        best_seller.append(images.find('img', alt='Best Sellers').get('alt'))

200
200


In [144]:
bestseller

[<img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/20-01w3-Icon-BestSeller-VUS-Home.svg" title="Best Sellers"/>,
 <img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/20-01w3-Icon-BestSeller-VUS-Home.svg" title="Best Sellers"/>,
 <img alt="Cruelty Free" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/icon-crueltyfree.svg" title="Cruelty Free"/>,
 <img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/20-01w3-Icon-BestSeller-VUS-Home.svg" title="Best Sellers"/>,
 <img alt="Cruelty Free" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/icon-crueltyfree.svg" title="Cruelty Free"/>,
 <img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/20-01w3-Icon-BestSeller-VUS-Home.svg" title="Best Sellers"/>,
 <img alt="Best Sellers" class="product-badge-icon" src="https://c

In [146]:
best_seller

['Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers',
 'Best Sellers']

In [135]:
names

['PURITO - Pure Vitamin C Serum - 60ml',
 'iUNIK - Tea Tree Relief Serum - 15ml',
 'SOME BY MI - Galactomyces Pure Vitamin C Glow Serum - 30ml',
 'SOME BY MI - AHA-BHA-PHA 30 Days Miracle Serum - 50ml',
 'Haruharu WONDER - Black Rice Facial Oil - 10ml',
 'iUNIK - Propolis Vitamin Synergy Serum - 50ml',
 'SOME BY MI - Yuja Niacin Blemish Care Serum - 50ml',
 "TIA'M - My Signature Red C Serum - 30ml",
 'Isntree - Clear Skin 8% AHA Essence - 100ml',
 'iUNIK - Propolis Vitamin Synergy Serum - 15ml',
 'Haruharu WONDER - Black Rice Hyaluronic Anti-wrinkle Serum - 50ml',
 "TIA'M - Vita B3 Source - 40ml"]

In [124]:
for euros in price:
        priceeuros.find('span', class_='price').text

9.99€
5.29€
10.69€
13.39€
4.39€
12.59€
12.59€
7.69€
9.39€
5.79€
10.59€
7.59€


In [122]:
cost

'7.59€'

In [43]:
for products in name:
    print(products.text)

iUNIK - Tea Tree Relief Serum - 50ml
COSRX - Advanced Snail 96 Mucin Power Essence 100ml
BEAUTY OF JOSEON - Repair Serum : Ginseng + Snail Mucin - 30ml
Krave - Great Barrier Relief - 45ml
BEAUTY OF JOSEON - Glow Serum : Propolis + Niacinamide - 30ml
BEAUTY OF JOSEON - Ginseng Essence Water - 150ml(5 fl.oz.)
PURITO - Centella Unscented Serum - 60ml
iUNIK - Black Snail Restore Serum
Rohto Mentholatum - Rohto Melano CC Concentration Measures Essence
Isntree - Hyaluronic Acid Water Essence - 50ml
BEAUTY OF JOSEON - Calming Serum : Green tea + Panthenol - 30ml
COSRX - BHA Blackhead Power Liquid 100ml


In [57]:
bestseller = soup.find_all('div', class_='product-badge product-badge-bestseller')

In [64]:
bestseller

[<div class="product-badge product-badge-bestseller" style="order:20">
 <img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/20-01w3-Icon-BestSeller-VUS-Home.svg" title="Best Sellers"/>
 <span class="product-badge-text"></span>
 </div>,
 <div class="product-badge product-badge-bestseller" style="order:20">
 <img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/20-01w3-Icon-BestSeller-VUS-Home.svg" title="Best Sellers"/>
 <span class="product-badge-text"></span>
 </div>,
 <div class="product-badge product-badge-bestseller" style="order:20">
 <img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/images/webicons/20-01w3-Icon-BestSeller-VUS-Home.svg" title="Best Sellers"/>
 <span class="product-badge-text"></span>
 </div>,
 <div class="product-badge product-badge-bestseller" style="order:20">
 <img alt="Best Sellers" class="product-badge-icon" src="https://cdn.stylevana.com/

In [81]:
for images in bestseller:
    print(images.find('img', alt='Best Sellers').get('alt'))

Best Sellers
Best Sellers
Best Sellers
Best Sellers
Best Sellers
Best Sellers
Best Sellers
Best Sellers
Best Sellers
Best Sellers
Best Sellers


In [ ]:
title = text.get('title', 'No title attribute')

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [37]:
essence = []

In [38]:
for link in essence_url[0:756]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name = soup.find('h1', class_='product-name-h1').text
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        price = (soup.find('span', class_='price').text.strip())
        best_seller = soup.find(class_="product-badge-icon").get('title')
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        details = 'no info'
        ingredients = 'no info'
        price = 'no info'
        best_seller = 'no'
        
    essence_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients,
    'price': price,
    'best-seller': best_seller}
    
    essence.append(essence_info)

In [39]:
print(len(essence))

756


## 3. PASARLO A DATAFRAME

In [40]:
essence_df = pd.DataFrame(essence)
essence_df

,name,details,ingredients,price,best-seller
0,iUNIK - Tea Tree Relief Serum - 50ml,"Usage: Day, Night\n\n\nProduct Type: Essence &...",[],9.09€,Best Sellers
1,COSRX - Advanced Snail 96 Mucin Power Essence ...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Snail Secretion Filtrate, Betaine, Butylene Gl...",11.89€,Best Sellers
2,BEAUTY OF JOSEON - Ginseng Essence Water - 150...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Panax ginseng root water, Butylene glycol, Gly...",10.59€,Best Sellers
3,BEAUTY OF JOSEON - Repair Serum : Ginseng + Sn...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[],9.29€,Best Sellers
4,BEAUTY OF JOSEON - Glow Serum : Propolis + Nia...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[],9.79€,Best Sellers
...,...,...,...,...,...
751,Dr.Ci:Labo - Super White 377 VC Essence - 18g,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Water, neopentyl glycol dicaprate, pentaerythr...",78.59€,Cruelty Free
752,Kose - DECORTE - Hydration Boosting Serum - 60ml,no info,no info,no info,no
753,Bffect - Granactive Retinoid Emulsion 2% + GAB...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Aqua, Propanediol, Simmondsia Chinensis (Jojob...",14.19€,Cruelty Free
754,TAKAMI - Skin Peel - 30ml,no info,no info,no info,no


In [41]:
essence_df = pd.DataFrame(essence)
essence_df

,name,details,ingredients,price,best-seller
0,iUNIK - Tea Tree Relief Serum - 50ml,"Usage: Day, Night\n\n\nProduct Type: Essence &...",[],9.09€,Best Sellers
1,COSRX - Advanced Snail 96 Mucin Power Essence ...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Snail Secretion Filtrate, Betaine, Butylene Gl...",11.89€,Best Sellers
2,BEAUTY OF JOSEON - Ginseng Essence Water - 150...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Panax ginseng root water, Butylene glycol, Gly...",10.59€,Best Sellers
3,BEAUTY OF JOSEON - Repair Serum : Ginseng + Sn...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[],9.29€,Best Sellers
4,BEAUTY OF JOSEON - Glow Serum : Propolis + Nia...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...",[],9.79€,Best Sellers
...,...,...,...,...,...
751,Dr.Ci:Labo - Super White 377 VC Essence - 18g,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Water, neopentyl glycol dicaprate, pentaerythr...",78.59€,Cruelty Free
752,Kose - DECORTE - Hydration Boosting Serum - 60ml,no info,no info,no info,no
753,Bffect - Granactive Retinoid Emulsion 2% + GAB...,"Cruelty Free: Yes\n\n\nUsage: Day, Night\n\n\n...","Aqua, Propanediol, Simmondsia Chinensis (Jojob...",14.19€,Cruelty Free
754,TAKAMI - Skin Peel - 30ml,no info,no info,no info,no


In [24]:
essence_df[essence_df['price'] == 'no info']

,name,details,ingredients,price,best-seller
1,ISOI - Acni Dr. 1st Control Essence - 50ml,no info,no info,no info,no
15,iUNIK - Tea Tree Relief Serum - 15ml,no info,no info,no info,no
17,SOME BY MI - Yuja Niacin Blemish Care Serum - ...,no info,no info,no info,no
19,Haruharu WONDER - Black Rice Facial Oil - 10ml,no info,no info,no info,no
21,SOME BY MI - Galactomyces Pure Vitamin C Glow ...,no info,no info,no info,no
...,...,...,...,...,...
745,Jealousness - Gold Helichrysum Water Essence -...,no info,no info,no info,no
746,HABA - Whitening Squalane,no info,no info,no info,no
747,BOTANIC HEAL BOH - Derma Intensive Energy Esse...,no info,no info,no info,no
748,LabNo - Lifted Idebenone Serum - 50ml,no info,no info,no info,no


In [42]:
essence = essence_df.to_csv('../data_/essence.csv')